<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD-GA MDA

FAST-OAD-GA is an add-on package for [FAST-OAD framework](https://github.com/fast-aircraft-design/FAST-OAD) for performing rapid Overall Aircraft Design in the category General Aviation (GA). The computational core of FAST-OAD being based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the Beechcraft Duchess. In `workdir/`, we store files generated or modified by FAST-OAD.

In [1]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path
DATA_FOLDER_PATH = 'data'
WORK_FOLDER_PATH = 'workdir'
OUTPUT_FOLDER_PATH = 'output'

# Clear work folder
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)
os.mkdir(WORK_FOLDER_PATH)

# Define files
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'MDA.yml')
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, 'beechcraft_76.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Failed to import module fastga.models.geometry.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests.test_openmdao_engine.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.geometry.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.performances.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_beechcraft_76.py
Failed to import

After defining a configuration file name, we can ask FAST-GA to generate a default configuration file based on the default OAD model implemented in the framework:

In [2]:
api_cs23.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

INFO    : Sample configuration written in workdir\MDA.yml


You can now checkout the generated [configuration file](./workdir/MDA.toml). In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [3]:
api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Problem inputs written in C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\notebooks\Lucas\workdir\problem_inputs.xml


'C:\\Users\\Lucas\\FAST-OAD-GA\\FAST-GA\\src\\fastga\\notebooks\\Lucas\\workdir\\problem_inputs.xml'

You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [ ]:
api_cs25.list_variables(CONFIGURATION_FILE)

Another useful feature is to list the modules of the model defined in the configuration file:

In [ ]:
api_cs25.list_systems(CONFIGURATION_FILE)



.Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api_cs25.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

## 2. Running your first MDA

### Beechcraft
Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

In [4]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:478: RuntimeWarning: divide by zero encountered in true_divide
  (radius_max - radius) / radius * math.sqrt(1 + (omega * radius / (v_inf + v_i)) ** 2.0)
c:\users\lucas\fast-oad-ga\venvtest\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:358: UserWarning: Propeller not working in propulsive mode!
  warnings.warn("Propeller not working in propulsive mode!")
c:\users\lucas\fast-oad-ga\venvtest\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\Lucas\FAST


aircraft_sizing


NL: NLBGS 1 ; 180255604 1
NL: NLBGS 2 ; 3458708.22 0.0191877986
NL: NLBGS 3 ; 2905909.77 0.0161210509
NL: NLBGS 4 ; 996230.408 0.00552676526
NL: NLBGS 5 ; 536410.322 0.00297583161
NL: NLBGS 6 ; 99328.0186 0.000551039838
NL: NLBGS 7 ; 55394.9363 0.000307313255
NL: NLBGS 8 ; 51657.525 0.000286579302
NL: NLBGS 9 ; 28317.3197 0.000157095364
NL: NLBGS 10 ; 8667.2377 4.80830417e-05
NL: NLBGS Converged


INFO    : Computation finished after 655.21 seconds
INFO    : Problem outputs written in C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\notebooks\Lucas\workdir\problem_outputs.xml


Let's save these results. We will use them in the next workbook, that shows some post-processing utilities.

In [6]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
MDA_OUTPUT_FILE = pth.join(OUTPUT_FOLDER_PATH, 'problem_outputs_mda_be76.xml')
shutil.copy(OUTPUT_FILE, MDA_OUTPUT_FILE)

'output\\problem_outputs_mda_be76.xml'

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [ ]:
api_cs25.variable_viewer(OUTPUT_FILE)